In [63]:
from datasets import load_dataset
from openai import OpenAI
from transformers import AutoTokenizer

In [10]:

aime = load_dataset("AI-MO/aimo-validation-aime", split="train")
amc = load_dataset("AI-MO/aimo-validation-amc", split="train")
math4 = load_dataset("AI-MO/aimo-validation-math-level-4", split="train")
math5 = load_dataset("AI-MO/aimo-validation-math-level-5", split="train")

In [ ]:
aime, amc, math4, math5

In [ ]:
d = math4[0]
print(d["id"], end="\n🔥\n")
print(d["problem"], end="\n🔥\n")
print(d["answer"], end="\n🔥\n")

In [ ]:
r"You are an expert mathematician, solve the problem ad put the answer in \boxed{}"

In [67]:
tokenizer = AutoTokenizer.from_pretrained("casperhansen/deepseek-r1-distill-qwen-1.5b-awq")

In [ ]:
# match (add_generation_prompt, continue_final_message)
# case (True, True): invalid, raise error
# case (True, False): will add nex turn's <start>user\nhello<end><start>assistant\n
# case (False, False): will end normally <start>user\nhello<end>
# case (False, True): will not include end token <start>user\nhello

tokens = tokenizer.apply_chat_template(
    [
        {
            "role": "system",
            "content": """be good""",
        },
        {
            "role": "user",
            "content": "aaa",
        },
    ],
    # add_generation_prompt=True,
    # continue_final_message=True,
    tokenize=False,


)
tokens

In [ ]:
client = OpenAI(base_url="http://localhost:8000/v1", api_key="-")
completion = client.chat.completions.create(
    model="casperhansen/deepseek-r1-distill-qwen-1.5b-awq",
    messages=[
        {
            "role": "system",
            "content": """You are an expert mathematician and programmer.""",
        },
        {
            "role": "user",
            "content": "Make a python code that solves this question: Mr. Madoff invests 1000 dollars in a fund that compounds annually at a constant interest rate.  After three years, his investment has grown to 1225 dollars.  What is the annual interest rate, as a percentage?  (Round your answer to the nearest integer.)",
        }
    ],
    #  messages=[
    #     {
    #         "role": "system",
    #         "content": r"你是一位数学专家，请解答下面的问题并将答案放在 \boxed{} 内。请用中文提供答案。",
    #     },
    #     {
    #         "role": "user",
    #         "content": "二次多项式 $P(x)$ 和 $Q(x)$ 的首项系数分别为 $2$ 和 $-2,$。两个多项式的图形都经过两点 $(16,54)$ 和 $(20,53)。求 $P(0) + Q(0)。$",
    #     },
    # ],
    temperature=0.6,
    stream=True,
    max_completion_tokens=10000,
)
for chunk in completion:
    print(chunk.choices[0].delta.content, end="")

In [ ]:
client = OpenAI(base_url="http://localhost:8000/v1", api_key="-")
prompt = tokenizer.apply_chat_template(
    [
        {
            "role": "system",
            "content": "you are an expert mathematician",
        },
        {
            "role": "user",
            "content": "Calculate the sum of all primes under 100.",
        },
        {
            "role": "assistant",
            "content": """<think>""",
        },
    ],
    tokenize=False,
    # add_generation_prompt=True,
    continue_final_message=True,
)
print(prompt, end="")
completion = client.completions.create(
    model="casperhansen/deepseek-r1-distill-qwen-1.5b-awq",
    prompt=prompt,
    temperature=0.6,
    stream=True,
    max_tokens=10000,
)
for chunk in completion:
    print(chunk.choices[0].text, end="")